In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from copy import deepcopy

In [2]:
interval_var = '5min'
symbol='BTC'

In [3]:
df = pd.read_csv('four_years.csv')

In [4]:
df

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,20180101,0,13863.14,13889.00,12952.50,13480.01,5208
1,20180102,0,13480.00,15275.00,13005.01,14781.51,10444
2,20180103,0,14781.52,15400.00,14628.00,15098.14,8382
3,20180104,0,15098.23,15400.00,14230.00,15144.99,9077
4,20180105,0,15145.00,17178.00,14824.05,16960.01,9286
...,...,...,...,...,...,...,...
1458,20211229,0,47542.20,48149.58,46094.02,46471.24,20337
1459,20211230,0,46471.24,47926.15,45938.44,47122.08,27411
1460,20211231,0,47122.09,48574.70,45650.00,46211.24,19017
1461,20220101,0,46211.24,47967.12,46205.00,47733.43,9472


In [5]:
df=df.drop('<TIME>',axis=1)


In [16]:
# Find slopes of price line 

def FindSlopes(prices, n):    
    slopes_array = [0]*(n-1)
    x = np.array(range(n))
    x_sc = (x - x.min())/(x.max() - x.min())
    x_sc = sm.add_constant(x_sc)
    for j in range(n,len(prices)+1):
        y = prices[j-n:j]
        y_sc = (y - y.min())/(y.max() - y.min())
        model = sm.OLS(y_sc,x_sc)
        results = model.fit()
        slopes_array.append(results.params[-1])
    slopes_array = np.rad2deg(np.arctan(np.array(slopes_array)))
    return slopes_array

In [21]:
# Find price channel

def FindChannel(high, low, close, lenght):
    channel_max = high.rolling(lenght).max()
    channel_min = low.rolling(lenght).min()
    channel_pos = (close - channel_min) / (channel_max - channel_min)
    return channel_pos

In [22]:
# generate data frame with all needed data

def PrepareDF(DF):
    df = DF.iloc[:,[0,1,2,3,4,5]]
    df.columns = ["date","open","high","low","close","volume"]
    
    print(df)

    
    df.set_index('date')
    
    print(df)
    
    df.reset_index()

    
    df['slope'] = FindSlopes(df['close'],5)
    df['position_in_channel'] = FindChannel(df['high'], df['low'], df['close'], 14)
    df = df.set_index('date')
    df = df.reset_index()
    return(df)

prepared_df = PrepareDF(df)
print(prepared_df)

          date      open      high       low     close  volume
0     20180101  13863.14  13889.00  12952.50  13480.01    5208
1     20180102  13480.00  15275.00  13005.01  14781.51   10444
2     20180103  14781.52  15400.00  14628.00  15098.14    8382
3     20180104  15098.23  15400.00  14230.00  15144.99    9077
4     20180105  15145.00  17178.00  14824.05  16960.01    9286
...        ...       ...       ...       ...       ...     ...
1458  20211229  47542.20  48149.58  46094.02  46471.24   20337
1459  20211230  46471.24  47926.15  45938.44  47122.08   27411
1460  20211231  47122.09  48574.70  45650.00  46211.24   19017
1461  20220101  46211.24  47967.12  46205.00  47733.43    9472
1462  20220102  47733.43  47990.00  46633.36  47299.07    6850

[1463 rows x 6 columns]
          date      open      high       low     close  volume
0     20180101  13863.14  13889.00  12952.50  13480.01    5208
1     20180102  13480.00  15275.00  13005.01  14781.51   10444
2     20180103  14781.52  1540

In [ ]:
def isLCC(DF,i):
    
    LCC=0
    
    if DF['close'][i]<=DF['close'][i+1] and DF['close'][i]<=DF['close'][i-1] and DF['close'][i+1]>DF['close'][i-1]:
        #найдено Дно
        LCC = i-1;
    return LCC

# MAX
def isHCC(DF,i):

    HCC=0
    if DF['close'][i]>=DF['close'][i+1] and DF['close'][i]>=DF['close'][i-1] and DF['close'][i+1]<DF['close'][i-1]:
        #найдена вершина
        HCC = i;
    return HCC





#%%

def getMaxMinChannel(DF, n):
    maxx=0
    minn=0
    for i in range (0,n-1):
        if maxx<DF['high'][len(DF)-i]:
            maxx=DF['high'][len(DF)-i]
        if minn>DF['low'][len(DF)-i]:
            minn=DF['low'][len(DF)-i]
    return(maxx,minn)

In [19]:
print(prepared_df)

          date      open      high       low     close  volume       TR  \
0     20180101  13863.14  13889.00  12952.50  13480.01    5208      NaN   
1     20180102  13480.00  15275.00  13005.01  14781.51   10444  2269.99   
2     20180103  14781.52  15400.00  14628.00  15098.14    8382   772.00   
3     20180104  15098.23  15400.00  14230.00  15144.99    9077  1170.00   
4     20180105  15145.00  17178.00  14824.05  16960.01    9286  2353.95   
...        ...       ...       ...       ...       ...     ...      ...   
1458  20211229  47542.20  48149.58  46094.02  46471.24   20337  2055.56   
1459  20211230  46471.24  47926.15  45938.44  47122.08   27411  1987.71   
1460  20211231  47122.09  48574.70  45650.00  46211.24   19017  2924.70   
1461  20220101  46211.24  47967.12  46205.00  47733.43    9472  1762.12   
1462  20220102  47733.43  47990.00  46633.36  47299.07    6850  1356.64   

              ATR      slope  channel_max  channel_min  position_in_channel  
0             NaN   0

In [ ]:
prepared_df[:100]['close'].plot()

In [ ]:
prepared_df[:100]['slope'].plot()

In [ ]:
prepared_df[:100][{'channel_max', 'channel_min', 'close'}].plot()

In [ ]:
prepared_df[:100]['position_in_channel'].plot()

In [ ]:
#prepared_df = prepared_df[:2016]

In [ ]:
lend = len(prepared_df)

In [ ]:
prepared_df['hcc'] = [None] * lend
prepared_df['lcc'] = [None] * lend

In [ ]:
for i in range(4, lend - 1):
    if isHCC(prepared_df, i) > 0:
        prepared_df.at[i, 'hcc'] = prepared_df['close'][i]
    if isLCC(prepared_df, i) > 0:
        prepared_df.at[i, 'lcc'] = prepared_df['close'][i]
prepared_df[:15]

In [ ]:
prepared_df[:50]['close'].plot()

In [ ]:
### рисовалка

aa = prepared_df[:100]
aa=aa.reset_index()

#labels = ['close',"deal_o","deal_c"]
labels = ['close',"hcc","lcc","channel_max","channel_min"]

labels_line = ['--',"*-","*-","g-","r-"]


j=0
x = pd.DataFrame()
y = pd.DataFrame()
for i in labels:
    x[j]=aa['index']
    y[j]=aa[i]
    j=j+1

fig, (ax1, ax2,ax3) = plt.subplots(3, 1)

fig.suptitle('Deals')
fig.set_size_inches(20, 10)

for j in range(0,len(labels)):
    ax1.plot(x[j], y[j], labels_line[j])


ax1.set_ylabel('Price')
ax1.grid(True)

ax2.plot(x[0], aa['slope'], '.-') #EMA
ax3.plot(x[0], aa['position_in_channel'], '.-') #EMA

ax2.grid(True)
ax3.grid(True)

plt.show()


In [ ]:
position=0
deal=0
#85481.99999999968
stop_percent = 0.038
btc_profit_array = [[251,10], [400, 0]]
x, y, mx, mn = 0, 0, 0, 0

print('go')
prepared_df['deal_o']=[None]*lend
prepared_df['deal_c']=[None]*lend
prepared_df['earn']=[None]*lend

wait = 0
sum_sosat = 0

cst = 53
sm = [0]*cst

for i in range(4,lend-1):
    sm.append(deal)
    
    
    
    if wait > 0:
        wait -= 1
        continue

    mx = max(mx, deal)
    mn = min(mn, deal)
    prepared_df.at[i,'earn']=deal

    if position>0:
    # long
        if(prepared_df['close'][i]<stop_price):
            #stop loss
            wait = 1
            pred = deal
            deal=deal-(open_price-prepared_df['close'][i]) * position
            sum_sosat += pred-deal
            position=0
            #print(i, 'stop loss')
            prepared_df.at[i,'deal_c']=prepared_df['close'][i]

        else:
            temp_arr=deepcopy(proffit_array)
            for j in range(0,len(temp_arr)-1):
                delta=temp_arr[j][0]
                contracts=temp_arr[j][1]
                if(prepared_df['close'][i]>(open_price+delta)):
                    #print(proffit_array)
                    prepared_df.at[i,'deal_c']=prepared_df['close'][i]
                    position=position-contracts
                    deal=deal+(prepared_df['close'][i]-open_price)*contracts
                    del proffit_array[0]

    elif position<0:
    # short
        if(prepared_df['close'][i]>stop_price):
            #stop loss
            wait = 1
            pred = deal
            deal=deal + (prepared_df['close'][i]-open_price) * position
            sum_sosat += pred-deal
            position=0
            prepared_df.at[i,'deal_c']=prepared_df['close'][i]
            #print(i, 'stop loss')
        else:
            temp_arr=deepcopy(proffit_array)
            for j in range(0,len(temp_arr)-1):
                delta=temp_arr[j][0]
                contracts=temp_arr[j][1]
                if(prepared_df['close'][i]<(open_price-delta)):
                    #print(proffit_array)
                    prepared_df.at[i,'deal_c']=prepared_df['close'][i]
                    position=position+contracts
                    deal=deal+(open_price-prepared_df['close'][i])*contracts
                    del proffit_array[0]
    else:
        if sm[i+cst-4]-sm[i-4]<-10000:
            continue
        if(prepared_df['lcc'][i-1]!=None):
            # long
            if(prepared_df['position_in_channel'][i-1]<0.3):
                if((prepared_df['slope'][i-1])<-20):
                    prepared_df.at[i,'deal_o']=prepared_df['close'][i]
                    #print(i,'open long position')
                    proffit_array=deepcopy(btc_profit_array)
                    position=10
                    open_price=prepared_df['close'][i]
                    stop_price=prepared_df['close'][i]*(1-stop_percent)
        if(prepared_df['hcc'][i-1]!=None):
            # short
            if(prepared_df['position_in_channel'][i-1]>0.67):
                if((prepared_df['slope'][i-1])>22):
                    prepared_df.at[i,'deal_o']=prepared_df['close'][i]
                    #print(i,'open short position')
                    proffit_array=deepcopy(btc_profit_array)
                    position=-10
                    open_price=prepared_df['close'][i]
                    stop_price=prepared_df['close'][i]*(1+stop_percent)
    
#print(deal, stop_percent, profit)
if deal > mx:
    mx = deal
    x = stop_percent
    #y = profit

print(deal)
print('maximum: ',mx, x, y)
print('minimum: ',mn, x, y)
print('sosat: ', sum_sosat)

In [ ]:
36649.89999999991 0.038 203
45936.19999999988 0.038 206
45936.19999999988 0.038 209
45936.19999999988 0.038 212
45936.19999999988 0.038 215
70686.39999999985 0.038 218
71937.29999999981 0.038 221
69910.7999999998 0.038 224
72169.49999999983 0.038 227
72169.49999999983 0.038 230
72930.29999999984 0.038 233
73570.49999999988 0.038 236
80899.6999999998 0.038 239
84918.49999999977 0.038 242
85306.59999999974 0.038 245
85481.99999999968 0.038 248
85481.99999999968 0.038 251
68608.09999999976 0.038 254
48761.09999999979 0.038 257
25945.69999999985 0.038 260
26562.799999999843 0.038 263
27962.399999999907 0.038 266
27962.399999999907 0.038 269
27962.399999999907 0.038 272
27962.399999999907 0.038 275
27962.399999999907 0.038 278
28190.199999999895 0.038 281
28190.199999999895 0.038 284
28231.899999999878 0.038 287
30805.099999999875 0.038 290
32185.099999999875 0.038 293
32185.099999999875 0.038 296
32185.099999999875 0.038 299
74347.29999999967 0.038 302
61437.69999999975 0.038 305
63005.29999999977 0.038 308
63149.799999999814 0.038 311
63149.799999999814 0.038 314
59983.09999999983 0.038 317

In [ ]:
49005.699999999924 0.017 176
51960.099999999875 0.017 179
52322.899999999936 0.017 182
52322.899999999936 0.017 185
52322.899999999936 0.017 188
40049.79999999989 0.017 191
40073.89999999985 0.017 194
39677.79999999984 0.017 197
40040.19999999982 0.017 200
41350.29999999977 0.017 203
36273.599999999715 0.017 206
37263.99999999972 0.017 209
37263.99999999972 0.017 212
35920.69999999978 0.017 215
44104.19999999984 0.017 218
45355.0999999998 0.017 221
43328.59999999979 0.017 224
45587.299999999814 0.017 227
45719.799999999814 0.017 230
46480.59999999983 0.017 233
47120.79999999987 0.017 236
47433.19999999978 0.017 239
50358.399999999674 0.017 242
50746.49999999965 0.017 245
50921.89999999959 0.017 248
50921.89999999959 0.017 251
44851.49999999958 0.017 254
45530.29999999955 0.017 257

In [ ]:
print(deal)

In [ ]:
df = pd.read_csv(r'C:\Users\aidar\four_years.csv')
df=df.drop('<TIME>',axis=1)
df.columns = ["date","open","high","low","close","volume"]

first_index = 0#52*7
for first_index in range(0, 52*7*4, 52*7):
    print()
    print(2018+first_index//(52*7))
    prepared_df = df[first_index:first_index+52*7]
    prepared_df = prepared_df.reset_index()
    #print(prepared_df)

    s = deepcopy(prepared_df['close'])
    s = s.shift(1)
    prepared_df['delta'] = prepared_df['close'] - s
    prepared_df

    count = [[0,0] for i in range(7)]

    const = 2.5
    for i in range(1, len(prepared_df)):
        if prepared_df['delta'][i] < 0:
            if abs(100*(prepared_df['delta'][i]/prepared_df['close'][i-1]))>const:
                count[i%7][0] += 1
                #count[i%7][0]-=100*(prepared_df['delta'][i]/prepared_df['close'][i-1])
        else:
            if abs(100*(prepared_df['delta'][i]/prepared_df['close'][i-1]))>const:
                count[i%7][1] += 1
                #count[i%7][1]+=100*(prepared_df['delta'][i]/prepared_df['close'][i-1])

    def func(a):
        s = 10-len(str(a))
        print(' '*s, a, end='')

    print(' '*19, 'падение ',' '*4, 'рост ')
    a = []
    first_day_in_year = 0
    for i in range(len(count)):
        index = (i+7-first_day_in_year)%7
        x,y = count[index]
        #print(f'day {i+1}: ',*count[index], y/x)
        a.append([i+1, *count[index], y+x])
    a.sort(key=lambda x: x[3])
    for i in range(len(a)):
        print('day: ',end = '')
        for j in range(len(a[i])):
            func(a[i][j])
        print()